In [ ]:
import matplotlib.pyplot as plt
import torch
import cv2
import yaml
from torchvision import transforms
import numpy as np

from utils.datasets import letterbox
from utils.general import non_max_suppression_mask_conf

from detectron2.modeling.poolers import ROIPooler
from detectron2.structures import Boxes
from detectron2.utils.memory import retry_if_cuda_oom
from detectron2.layers import paste_masks_in_image

%matplotlib inline

### [Original Repository](https://github.com/WongKinYiu/yolov7)

### [Mask Repository](https://github.com/WongKinYiu/yolov7/tree/mask)

### [Segmentation Training](https://github.com/WongKinYiu/yolov7/tree/u7/seg)

### [Segmentation Weights](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x-seg.pt)

## Detectron2 Dependency
In order to use [`yolov7-mask`](https://github.com/WongKinYiu/yolov7/tree/mask) model for prediction little is required, however the [`u7`](https://github.com/WongKinYiu/yolov7/tree/u7/seg) branch containing training information for the [`yolov7-seg.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-seg.pt) model requires [`Detectron2`](https://github.com/facebookresearch/detectron2) installation. 

This additional requirement can be rather involved, as it requires matching [`CUDA`](https://developer.nvidia.com/cuda-downloads)/[`torch`](https://pytorch.org/get-started/locally/)/[`detectron`](https://detectron2.readthedocs.io/en/latest/tutorials/install.html) distributions, where PyTorch must be compiled with the current CUDA compiler version, as well as some additional [`protoc`](https://grpc.io/docs/protoc-installation/) versioning that typically isn't a hard requirement for these sorts of projects. These complications are, to my understanding, the reason the segmentation branch hasn't been merged into main.

Much of this complication was _seemingly_ deprecated with the release of Ultralytic's YOLOv8 that more easily supports training and validation of both segmentation and detection models.

In [ ]:
!python segment/train.py --data coco.yaml --batch 8 --weights '' --cfg yolov7-seg.yaml --epochs 300 --name yolov7-seg --img 640 --hyp hyp.scratch-high.yaml


In [ ]:
YOLOTYPE = "instance"
TRAIN = False
MULTIGPU = True
NAME = ''
IMG_SIZE = 640
BATCH_SIZE = 4 #multiple of number of GPUs
EPOCHS = 800

if YOLOTYPE in ['instance','mask']:
    hyp_yaml_path = 'data/hyp.scratch.custom.yaml'
    train_yaml_path = 'cfg/training/yolov7-seg.yaml'
    weight_path = 'model_weights/yolov7-seg.pt'
elif YOLOTYPE in ['xl','e6e']:
    hyp_yaml_path = 'data/hyp.scratch.p6.yaml'
    train_yaml_path = 'cfg/training/yolov7-e6e.yaml'
    weight_path = 'yolov7-e6.pt'
elif YOLOTYPE in ['l','w6']:
    hyp_yaml_path = 'data/hyp.scratch.p6.yaml'
    train_yaml_path = 'cfg/training/yolov7-w6.yaml'
    weight_path = 'yolov7-w6.pt'    
elif YOLOTYPE in ['','standard','v7']:
    hyp_yaml_path = 'data/hyp.scratch.p5.yaml'
    train_yaml_path = 'cfg/training/yolov7.yaml'
    weight_path = 'yolov7.pt'
else:
    raise ArgumentError(f'YOLOTYPE {YOLOTYPE} not recognized')
    
if TRAIN is True: #overwrite previous declarations for pretrained version
    hyp_yaml_path = 'data/hyp.scratch.custom.yaml'
    weight_path = weight_path.replace('.pt','_training.pt')

if not os.path.exists(weight_path): # if chosen weights aren't available, attempt to download 
    os.system(f"wget -O {weight_path} 'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/{weight_path.split('/')[-1]}'")

In [ ]:
if TRAIN is True:
    if MULTIGPU is True:
        command_str = f"""
            python 
            segment/train.py 
            --workers 8 
            --device 0 
            --batch-size {BATCH_SIZE}
            --data data/{NAME}.yaml  
            --img {IMG_SIZE} 
            --cfg {train_yaml_path} 
            --weights {weight_path}
            --name {NAME}
            --hyp {hyp_yaml_path} 
            --epochs {EPOCHS}
        """
    else:
        command_str = f"""
            python -m 
            torch.distributed.launch 
            --nproc_per_node 4 
            --master_port 9527 

            segment/train.py 
            --workers 8 
            --device 0,1,2,3 
            --batch-size {BATCH_SIZE}
            --data data/{NAME}.yaml  
            --img {IMG_SIZE} 
            --cfg {train_yaml_path} 
            --weights {weight_path}
            --name {NAME}
            --hyp {hyp_yaml_path} 
            --epochs {EPOCHS}
        """

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
with open(hyp_yaml_path) as hyp_yaml:
    hyp = yaml.load(hyp_yaml, Loader=yaml.FullLoader)

weights = torch.load(weight_path)
model = weights['model']
model = model.half().to(device)
_ = model.eval()

In [ ]:
IMAGE_PATHS = ['./horses.jpg']
IMG_SIZE = 640

def process_img_as_tensor(image_path):
    image = cv2.imread(image_path)
    image = letterbox(image, IMG_SIZE, stride=64, auto=True)[0]
    image_ = image.copy()
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.asarray([image.numpy()]))
    image = image.to(device)
    image = image.half()
    return image

def draw_img_from_output(output):
    inf_out, train_out, attn, mask_iou, bases, sem_output = output['test'], output['bbox_and_cls'], output['attn'], output['mask_iou'], output['bases'], output['sem']
    bases = torch.cat([bases, sem_output], dim=1) #concat torch tensor?
    nb, _, height, width = image.shape #get tensor dimensions
    names = model.names #class labels?
    pooler_scale = model.pooler_scale
    pooler = ROIPooler(output_size=hyp['mask_resolution'], scales=(pooler_scale,), sampling_ratio=1, pooler_type='ROIAlignV2', canonical_level=2)
    output, output_mask, output_mask_score, output_ac, output_ab = non_max_suppression_mask_conf(inf_out, attn, bases, pooler, hyp, conf_thres=0.25, iou_thres=0.65, merge=False, mask_iou=None)
    pred, pred_masks = output[0], output_mask[0]
    base = bases[0]
    bboxes = Boxes(pred[:, :4])
    original_pred_masks = pred_masks.view(-1, hyp['mask_resolution'], hyp['mask_resolution'])
    pred_masks = retry_if_cuda_oom(paste_masks_in_image)( original_pred_masks, bboxes, (height, width), threshold=0.5)
    pred_masks_np = pred_masks.detach().cpu().numpy()
    pred_cls = pred[:, 5].detach().cpu().numpy()
    pred_conf = pred[:, 4].detach().cpu().numpy()
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    nbboxes = bboxes.tensor.detach().cpu().numpy().astype(np.int)
    pnimg = nimg.copy()
    for one_mask, bbox, cls, conf in zip(pred_masks_np, nbboxes, pred_cls, pred_conf):
        if conf < 0.25:
            continue
        color = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]


        pnimg[one_mask] = pnimg[one_mask] * 0.5 + np.array(color, dtype=np.uint8) * 0.5
        pnimg = cv2.rectangle(pnimg, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
        #label = '%s %.3f' % (names[int(cls)], conf)
        #t_size = cv2.getTextSize(label, 0, fontScale=0.5, thickness=1)[0]
        #c2 = bbox[0] + t_size[0], bbox[1] - t_size[1] - 3
        #pnimg = cv2.rectangle(pnimg, (bbox[0], bbox[1]), c2, color, -1, cv2.LINE_AA)  # filled
        #pnimg = cv2.putText(pnimg, label, (bbox[0], bbox[1] - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)  
    return pnimg

def show_drawn_image(drawn_image):
    plt.figure(figsize=(8,8))
    plt.axis('off')
    plt.imshow(drawn_image)
    plt.show()
    return

outputs = list()
for image_path in IMAGE_PATHS:
    image = process_img_as_tensor(image_path)
    outputs.append(model(image))
    
if YOLOTYPE in ['mask','instance']:
    for output in outputs:
        drawn_image = draw_img_from_output(output)
        show_drawn_image(drawn_image)        

In [3]:
%cd 2019_all
!ls

[WinError 2] The system cannot find the file specified: '2019_all'
C:\Users\Nick\Documents\Classes\Current\Research\NASA\2019_all
spots_2019-01-01_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-02_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-03_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-04_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-05_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-06_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-07_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-08_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-09_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-10_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-11_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-12_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-13_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-14_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-15_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-16_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-17_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-18_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-19_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-20_T0_B20_RBN_WSPR_PSK_.jpg
spots_2019-01-21_T0_B20_RBN_WS

In [5]:
import os
files = [file for file in os.listdir()]
print(files[0:10])

['spots_2019-01-01_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-02_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-03_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-04_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-05_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-06_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-07_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-08_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-09_T0_B20_RBN_WSPR_PSK_.jpg', 'spots_2019-01-10_T0_B20_RBN_WSPR_PSK_.jpg']


In [6]:
import shutil
for file in files:
    new_file = file.split('_')[1] + '.jpg'
    shutil.move(file, new_file)

In [7]:
!ls

2019-01-01.jpg
2019-01-02.jpg
2019-01-03.jpg
2019-01-04.jpg
2019-01-05.jpg
2019-01-06.jpg
2019-01-07.jpg
2019-01-08.jpg
2019-01-09.jpg
2019-01-10.jpg
2019-01-11.jpg
2019-01-12.jpg
2019-01-13.jpg
2019-01-14.jpg
2019-01-15.jpg
2019-01-16.jpg
2019-01-17.jpg
2019-01-18.jpg
2019-01-19.jpg
2019-01-20.jpg
2019-01-21.jpg
2019-01-22.jpg
2019-01-23.jpg
2019-01-24.jpg
2019-01-25.jpg
2019-01-26.jpg
2019-01-27.jpg
2019-01-28.jpg
2019-01-29.jpg
2019-01-30.jpg
2019-01-31.jpg
2019-02-01.jpg
2019-02-02.jpg
2019-02-03.jpg
2019-02-04.jpg
2019-02-05.jpg
2019-02-06.jpg
2019-02-07.jpg
2019-02-08.jpg
2019-02-09.jpg
2019-02-10.jpg
2019-02-11.jpg
2019-02-12.jpg
2019-02-13.jpg
2019-02-14.jpg
2019-02-15.jpg
2019-02-16.jpg
2019-02-17.jpg
2019-02-18.jpg
2019-02-19.jpg
2019-02-20.jpg
2019-02-21.jpg
2019-02-22.jpg
2019-02-23.jpg
2019-02-24.jpg
2019-02-25.jpg
2019-02-26.jpg
2019-02-27.jpg
2019-02-28.jpg
2019-03-01.jpg
2019-03-02.jpg
2019-03-03.jpg
2019-03-04.jpg
2019-03-05.jpg
2019-03-06.jpg
2019-03-07.jpg
2019-03-08

In [11]:
import cv2
import numpy as np

IMG_SIZE = (719, 301, 3)
POSITIONS = ['xmin','ymin','xmax','ymax']
CROP_DIFS = (125, 844, 153, 454)

def crop_plotted_img(img, grayscale=False):
    assert img.shape == (600,1000,3)
    img = img[CROP_DIFS[2]:CROP_DIFS[3],CROP_DIFS[0]:CROP_DIFS[1]]
    img_patch = img[0:100,0:200]
    img_patch[np.where((img_patch > [30,30,30]).all(axis=-1))] = [0,0,0]
    if grayscale: img = np.expand_dims(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), axis=2)
    return img

for file in os.listdir():
    img = cv2.imread(file)
    img = crop_plotted_img(img)
    cv2.imwrite(file, img)

In [12]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key='')
project = rf.workspace("tid-detection").project("ionospheresegmentation")
dataset = project.version(11).download("yolov7")


loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Extracting Dataset Version Zip to IonosphereSegmentation-11 in yolov7pytorch:: 100%|██████████| 1628/1628 [00:01<00:00, 1597.95it/s]


In [13]:
!ls

2019-01-01.jpg
2019-01-02.jpg
2019-01-03.jpg
2019-01-04.jpg
2019-01-05.jpg
2019-01-06.jpg
2019-01-07.jpg
2019-01-08.jpg
2019-01-09.jpg
2019-01-10.jpg
2019-01-11.jpg
2019-01-12.jpg
2019-01-13.jpg
2019-01-14.jpg
2019-01-15.jpg
2019-01-16.jpg
2019-01-17.jpg
2019-01-18.jpg
2019-01-19.jpg
2019-01-20.jpg
2019-01-21.jpg
2019-01-22.jpg
2019-01-23.jpg
2019-01-24.jpg
2019-01-25.jpg
2019-01-26.jpg
2019-01-27.jpg
2019-01-28.jpg
2019-01-29.jpg
2019-01-30.jpg
2019-01-31.jpg
2019-02-01.jpg
2019-02-02.jpg
2019-02-03.jpg
2019-02-04.jpg
2019-02-05.jpg
2019-02-06.jpg
2019-02-07.jpg
2019-02-08.jpg
2019-02-09.jpg
2019-02-10.jpg
2019-02-11.jpg
2019-02-12.jpg
2019-02-13.jpg
2019-02-14.jpg
2019-02-15.jpg
2019-02-16.jpg
2019-02-17.jpg
2019-02-18.jpg
2019-02-19.jpg
2019-02-20.jpg
2019-02-21.jpg
2019-02-22.jpg
2019-02-23.jpg
2019-02-24.jpg
2019-02-25.jpg
2019-02-26.jpg
2019-02-27.jpg
2019-02-28.jpg
2019-03-01.jpg
2019-03-02.jpg
2019-03-03.jpg
2019-03-04.jpg
2019-03-05.jpg
2019-03-06.jpg
2019-03-07.jpg
2019-03-08

In [18]:
%cd ..
%cd 2017_2019v2

C:\Users\Nick\Documents\Classes\Current\Research\NASA
C:\Users\Nick\Documents\Classes\Current\Research\NASA\2017_2019v2


In [19]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3A58-B400

 Directory of C:\Users\Nick\Documents\Classes\Current\Research\NASA\2017_2019v2

11/27/2022  07:24 PM    <DIR>          .
11/27/2022  07:24 PM    <DIR>          ..
11/27/2022  07:24 PM               333 data.yaml
11/27/2022  07:24 PM               161 README.dataset.txt
11/27/2022  07:24 PM               836 README.roboflow.txt
11/27/2022  07:24 PM    <DIR>          test
11/27/2022  07:24 PM    <DIR>          train
11/27/2022  07:24 PM    <DIR>          valid
               3 File(s)          1,330 bytes
               5 Dir(s)   6,400,946,176 bytes free


In [24]:
import shutil

for directory in ['train/','test/','valid/']:
    for subdirectory in ['images/','labels/']:
        path = os.path.join(directory, subdirectory)
        for file in os.listdir(path):
            filepath = os.path.join(path, file)
            ext = filepath.split('.')[-1]
            newfilepath = filepath.split('_')[0] + '.' + ext
            shutil.move(filepath, newfilepath)